In [ ]:
import torch
from sklearn.model_selection import train_test_split
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import lightning.pytorch as pl

# Load in the Tenosrs

In [2]:
X_nn, y_nn = torch.load("../../data/emulator/nn_data/X_nn.pt"), torch.load("../../data/emulator/nn_data/y_nn.pt")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_nn, y_nn, random_state=22, train_size=.8)

In [4]:
X_train.shape, X_test.shape

(torch.Size([187143, 40]), torch.Size([46786, 40]))

In [5]:
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_test, y_test)  

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=3)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=3)

# Build the model

In [6]:
class LitFeedForward(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        self.layer_2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.layer_1(x))
        x = self.layer_2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.sqrt(F.mse_loss(y_hat, y)) 
        self.log('val_loss', loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer


# Train the model

To view training in tensorboard:

`tensorboard --logdir lightning_logs`

In [8]:
input_dim = X_train.shape[1] 
hidden_dim = 64  
output_dim = 40  

model = LitFeedForward(input_dim, hidden_dim, output_dim)

trainer = pl.Trainer(max_epochs=10) 

trainer.fit(model, train_loader, val_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 2.6 K 
1 | layer_2 | Linear | 2.6 K 
-----------------------------------
5.2 K     Trainable params
0         Non-trainable params
5.2 K     Total params
0.021     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


# Rest will be run on a HPC

- TODO: Make a script to run on the hpcc.
